In [56]:
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np

In [57]:
def calc_proportion_es(prob1: float, prob2: float):

    return abs(proportion_effectsize(prob1, prob2))


# def calc_proportion_es_alt(conv1: float, conv2: float, prob1: float, prob2: float):

#     return 2 * asin(np.sqrt(conv1/nobs1)) - 2 * asin(np.sqrt(conv2/nobs2))


def calc_proportion_es_alt(conv1: float, conv2: float, prob1: float, prob2: float):

    return 2 * asin(np.sqrt(conv1/prob1)) - 2 * asin(np.sqrt(conv2/prob2))


def calc_continuous_es(mean_control: Union[float, int],
                       std_control: Union[float, int],
                       mean_test: Union[float, int],
                       std_test: Union[float, int]):

    return abs(effectsize_smd(mean_control,
                              std_control,
                              1e4,
                              mean_test,
                              std_test,
                              1e4)[0])


def calc_continuous_es_alt(mean_control: Union[float, int],
                           std_control: Union[float, int],
                           mean_test: Union[float, int],
                           std_test: Union[float, int]):

    effect_size = (mean_test - mean_control) / \
        ((std_control**2 + std_test**2) / 2) ** 0.5
    return effect_size


def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):

    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                           )
    return int(n * 2)


def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):

    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                           )
    return int(n * 2)


### Задача 1.

Продакт на главной mail.ru решил протестировать в рекомендательной ленте контента вместо карточек со статьями видеоплеер с короткими видео.

Нынешний таймспент на юзера в день в среднем равен 25 минут, а стандартная ошибка (SD) равна 156.

Мы предполагаем, что в новой версии таймспент на юзера в день изменится на 10%. Средний трафик 20000 человек в день.

Посчитайте сколько дней необходимо держать эксперимент при alpha = 5% и beta = 20% .

In [58]:
time_spend_control, time_spend_test = 25, 27.5
std_control, std_test = 156, 156 * 1.15
DAU = 20000

In [59]:
es_cont = calc_continuous_es(time_spend_control, std_control, time_spend_test, std_test)
res = calc_sample_size_continuous(es_cont) / DAU

es_cont, calc_sample_size_continuous(es_cont), res

(0.014870865944557932, 141971, 7.09855)

In [60]:
es_cont_alt = calc_continuous_es_alt(time_spend_control, std_control, time_spend_test, std_test)
res = calc_sample_size_continuous(es_cont_alt) / DAU

es_cont_alt, calc_sample_size_continuous(es_cont_alt), res

(0.014871423685692022, 141960, 7.098)

Вывод: длительность теста должна быть 7 дней

### Задача 2.

Наша продуктовая команда в ecommerce магазине планирует запустить тест, направленный на ускорение загрузки сайта.

Одна из основных метрик bounce rate в GA = 40%. Мы предполагаем, что при оптимизации сайта она изменится минимум на 20%.

Средний трафик 4000 человек в день. Посчитайте сколько нам нужно дней держать эксперимент при alpha = 5% и beta = 20%

In [62]:
brGA_1, brGA_2 = 0.40, 0.32
DAU = 4000

In [63]:
es_prop = calc_proportion_es(brGA_1, brGA_2)
calc_sample_size_proportion(es_prop)
res = calc_sample_size_proportion(es_prop) / DAU

es_prop, calc_sample_size_proportion(es_prop), res

(0.16690997264630925, 1126, 0.2815)

In [67]:
es_prop_alt = calc_proportion_es_alt(brGA_1, brGA_2, 1, 1)
calc_sample_size_proportion(es_prop_alt)
res = calc_sample_size_proportion(es_prop_alt) / DAU

es_prop, calc_sample_size_proportion(es_prop_alt), res

(0.16690997264630925, 1126, 0.2815)

Вывод: длительность теста должна быть не менне 0.28 дня (или 6 часов 45 минут 22 сек =)))))